## Introduction

Brainbox is a service that:

* hosts _Deciders_, representations of ML-services, such as Oobabooga or Automatic1111.
* provides a unified API access to the deciders
* starts and stops the deciders, managing the GPU load
* queues incoming tasks and executes them asynchonously, storing results in a database.

BrainBox is supposed to be constantly on a GPU machine and execute the tasks when it's not busy. 

Let's configure a local BrainBox and run some tasks on it. 

In [1]:
from kaia.brainbox import BrainBoxSettings, BrainBoxTestApi, BrainBoxTask
from kaia.brainbox.deciders.fake_image_generator import FakeImageDecider

#Dict of decider is needed to build a correspondence between the name and the type of the decider
deciders = {'FakeAutomatic1111': FakeImageDecider()}

#Setting contain various settings of everything ML-related, including the settings of the supported models. We can alter these settings
settings = BrainBoxSettings()
settings.brain_box_web_port = 8091

#This line boots up brainbox in a testmode. It will work as a real webserver, and will be brought down after the `with` scope.
with BrainBoxTestApi(deciders) as api:
    task = BrainBoxTask(id = BrainBoxTask.safe_id(), decider = 'FakeAutomatic1111', arguments = dict(prompt='Some prompt'), decider_method = None)
    result = api.execute(task)
    files = [api.download(filename) for filename in result]

To run the task, BrainBox determines the appropriate decider and runs einther `__call__` (if `decider_method` is None, which is a default), or a specified method. `arguments` are passed to this method.

FakeImageDecider, as well as Automatic1111, returns the list of files' names:

In [5]:
result

['938f0aef-2088-4c32-bef3-6b62c7b2db32.json',
 'fff11351-9fd5-4662-ab94-fc2840a18093.json',
 '64945b19-6ef0-4fb1-a050-679da981c459.json',
 'cff611bc-385d-42a5-84fd-0c890be38f8e.json']

`execute` method can also accept a list of tasks, in this case `result` will be an array, each element containing the result of the corresponding task.

The files are located within BrainBox cache, and Brainbox can be located on a different machine. This is why the files need to be downloaded. Let's view one of the downloaded files:

In [7]:
from kaia.infra import FileIO

FileIO.read_json(files[0])

{'prompt': 'Some prompt', 'option_index': 0, 'model': None}

We can also execute tasks asyncronously:

In [8]:
with BrainBoxTestApi(deciders) as api:
    task = BrainBoxTask(id = BrainBoxTask.safe_id(), decider = 'FakeAutomatic1111', arguments = dict(prompt='Some prompt'))
    api.add(task)
    print('Do something while Brainbox is executing out task')
    result = api.join(task)

result

Do something while Brainbox is executing out task


['abe91c29-4c80-48e8-8f3d-43541b958ae8.json',
 'b1912260-1762-4640-9f58-d14c2671b6c5.json',
 'fb2924cf-49f9-4d86-b8da-9c5509feec70.json',
 '7bdde1c2-fab7-47c1-88ce-08a3eefb657d.json']

We can poll the task status:

In [13]:
from pprint import pformat

with BrainBoxTestApi(deciders) as api:
    task = BrainBoxTask(id = BrainBoxTask.safe_id(), decider = 'FakeAutomatic1111', arguments = dict(prompt='Some prompt'))
    api.add(task)
    job_state = api.get_job(task.id)
    result = api.join(task)

print(pformat({key:value for key, value in job_state.__dict__.items() if not key.startswith('_')}))

{'accepted': False,
 'accepted_timestamp': None,
 'arguments': {'prompt': 'Some prompt'},
 'assigned': False,
 'assigned_timestamp': None,
 'back_track': None,
 'batch': None,
 'decider': 'FakeAutomatic1111',
 'decider_parameters': None,
 'dependencies': None,
 'error': None,
 'finished': False,
 'finished_timestamp': None,
 'id': 'id_ac967fb2ddba41a5866258c9de3d4516',
 'log': None,
 'method': None,
 'progress': None,
 'ready': False,
 'ready_timestamp': None,
 'received_timestamp': datetime.datetime(2024, 2, 13, 17, 35, 44, 172904),
 'result': None,
 'success': False}


## Warming up and cooling down

When decider warms up, it accepts a string parameter that may affect the decider. The most obvious use case for the parameter is a base model for Automatic1111 or Oobabooga. 

In this implementation, one cannot change the parameter of the running decider: it needs to be cooled down and then warmed up. Since the majority of time is anyways consumed to load the model, it's not a big problem.

A `Planer` is used to determine which deciders to warm up or cool down, and also which tasks should be executed on the currently-up deciders. `SimpleDecider` which is used in `BrainBoxTestApi`, selects the decider for longest-waiting task, warms the decider up, executes all tasks that match this decider, and then cools the decider down. Other planners with more sophisticated logic can be implemented. 

Let's see how parameters work in action

In [ ]:
test_api = BrainBoxTestApi(deciders)
with test_api as api:
    tasks = []
    for index in range(3):
        for model in ['a','b','c']:
            tasks.append(BrainBoxTask(
                id = BrainBoxTask.safe_id(), 
                decider = 'FakeAutomatic1111', 
                arguments = dict(prompt=f'{model}/{index}'), 
                decider_parameters=model))
    results = api.execute_tasks(tasks)
    files = [api.download(result[0]) for result in results]

In [ ]:
import pandas as pd

pd.DataFrame([FileIO.read_json(file) for file in files])

We can see that indeed the tasks were called with different models. 

## Media Libraries and tasks dependencies

In the on-premise hosting setup, we can't count on continuous availability of the GPU. Hence, a strategy can be to generate excess amount of content in advance. This scenario is supported in BrainBox with `Collector` decider and `MediaLibrary` format.

In [14]:
from kaia.brainbox.deciders.collector import Collector

deciders = {'FakeAutomatic1111': FakeImageDecider(), 'Collector': Collector()}

with BrainBoxTestApi(deciders) as api:
    tags = {}
    dependencies = {}
    for i in range(5):
        id = BrainBoxTask.safe_id()
        tags[id] = dict(index=i)
        dependencies[id] = id
        api.add(BrainBoxTask(id = id, decider = 'FakeAutomatic1111', arguments = dict(prompt='Some prompt')))

    collection_task = BrainBoxTask(id = BrainBoxTask.safe_id(), decider = 'Collector', dependencies = dependencies, arguments = dict(tags=tags))
    result = api.execute([collection_task])
    library = api.download(result[0])
           

What happens here? BrainBox processes key-value pairs in `dependencies` by assigning the result of the task with id `value` to an argument of the decider's method with the name `key`. In our case, `key` equls `value`, but in other scenarios it may happen that the result of one decider must be injected as a particular argument to other decider, hence the dictionary. 

`Collector` then assembles all the outputs into one zip file, a media library. This media library also keeps tags, assotiated with the files. Collector can process any decider's output as long as it's a list of files.

In [15]:
from kaia.brainbox import MediaLibrary

lib = MediaLibrary.read(library)
lib.to_df().head(5)

,index,option_index,filename,timestamp,job_id
0,0,0,a3898697-15b5-4d53-a438-75c43e003160.json,2024-02-13 17:36:33.058262,id_7137120cab0942b2bd461bd2fcaaf0d4
1,0,1,400cf936-2f49-4a73-925e-316df92b0056.json,2024-02-13 17:36:33.058262,id_7137120cab0942b2bd461bd2fcaaf0d4
2,0,2,e14d313c-ba92-4f2a-b016-a084114da333.json,2024-02-13 17:36:33.058262,id_7137120cab0942b2bd461bd2fcaaf0d4
3,0,3,9662d21d-91ba-4a1b-8531-109d835cebd1.json,2024-02-13 17:36:33.058262,id_7137120cab0942b2bd461bd2fcaaf0d4
4,1,0,12de8566-02a3-4e27-b758-22d75c67eec3.json,2024-02-13 17:36:33.058262,id_a3bf479042ef42abb1f5baf7fadce038


The files can be extracted from the library:

In [16]:
lib.records[0].get_content()

b'{\r\n "prompt": "Some prompt",\r\n "option_index": 0,\r\n "model": null\r\n}'

## TaskPack

The approach with `Collector` and `MediaLibrary` is very handy to generate content. Some additional syntax sugar was created to make this as easy as possible in the code. `BrainBoxTaskPack` allows you:
* to define intermediate tasks that are required to complete the main, `resulting_task`
* to define postprocessor that will be performed on the API side with the result: e.g., to download the file and to open it with `MediaLibrary.read`

It is important to remember that all the pack-related functionality is performed by API, not by the BrainBox web-server.

In [20]:
from kaia.brainbox import BrainBoxTaskPack, DownloadingPostprocessor

tasks = []
tags = {}
dependencies = {}
for i in range(5):
    id = BrainBoxTask.safe_id()
    tags[id] = dict(index=i)
    dependencies[id] = id
    tasks.append(BrainBoxTask(id = id, decider = 'FakeAutomatic1111', arguments = dict(prompt='Some prompt')))

collection_task = BrainBoxTask(id = BrainBoxTask.safe_id(), decider = 'Collector', dependencies = dependencies, arguments = dict(tags=tags))

pack = BrainBoxTaskPack(
    collection_task, 
    tuple(tasks),
    DownloadingPostprocessor(opener=MediaLibrary.read)
)

with BrainBoxTestApi(deciders) as api:
    lib = api.execute(pack)

lib.to_df().head()

,index,option_index,filename,timestamp,job_id
0,0,0,14875720-b786-4a26-864c-0259df9cade5.json,2024-02-13 17:39:37.815175,id_ce0a60370ee34a1fb4f04de4e2288e0a
1,0,1,0a041718-d5f5-4948-b0ac-367a2380ae8f.json,2024-02-13 17:39:37.815175,id_ce0a60370ee34a1fb4f04de4e2288e0a
2,0,2,3e65dd83-9c2e-4776-ad60-e52fb515ae6a.json,2024-02-13 17:39:37.815175,id_ce0a60370ee34a1fb4f04de4e2288e0a
3,0,3,dd0fc506-6a2a-423e-94bd-91dc2cd2f688.json,2024-02-13 17:39:37.815175,id_ce0a60370ee34a1fb4f04de4e2288e0a
4,1,0,9b232f19-19e0-478a-90c7-7401a4a5b090.json,2024-02-13 17:39:37.815175,id_fa1877425e41496f8fac96970c09f0e6


BrainBoxTaskPack is a handy way of representing complex task networks in your applications and then work with a single-liner `api.execute` to obtain the programmatical representation of the result in the application.

# Outro

In the following notebooks, installation instructions for the external models, used in BrainBox, are given. 

If you already have Oobaboga, Stable Diffusion or other models installed, you may need to configure the Deciders, providing in the settings the location, the path to environment's python, etc. If you choose to follow the supplied installation instructions, the default settings should work and you can start BrainBox with them just by running `kaia/brainbox/run_brain_box.py`

Also, you may want to check from time to time that the deciders still work. To do that, use `kaia_tests/test_brainbox/model_integration_tests.py`